In [1]:
import blocksci
import matplotlib.pyplot as plt
import matplotlib.ticker
import collections
import pandas as pd
import numpy as np
from pathlib import Path
%matplotlib inline

parser_data_directory = Path("/mnt/anal/config.json")
cluster_directory = Path("/mnt/anal/cluster/")
dumplings_directory = Path("/mnt/dumplings/")

chain = blocksci.Blockchain(str(parser_data_directory))

from typing import Tuple

def get_block_height_for_date(date: str) -> int:
    return chain.range(date)[0].height

def get_block_height_range(start: str, end: str) -> Tuple[int, int]:
    return get_block_height_for_date(start), get_block_height_for_date(end)

In [2]:
import json

ww1_false = set(json.loads(open("/mnt/anal/false_cjtxs_ww1.json").read()))
c = 0
for tx in ww1_false:
    tx = chain.tx_with_hash(tx)
    print(tx.hash)
    if tx.is_wasabi1_coinjoin:
        print(blocksci.CoinJoinType(tx.coinjoin_tag))
        for i in tx.inputs:
            print(f"is ww1 cj? {i.index} {i.spent_tx.is_wasabi1_coinjoin}")
        print(tx.mempool_space_link)
        c += 1
        if c > 2:
            break

57dfe22896ee38e71f064ccbe8fca668f5d82aa1d437736b513e4f3f8e55d845
CoinJoinType.WW1
is ww1 cj? 0 False
is ww1 cj? 1 False
is ww1 cj? 2 False
is ww1 cj? 3 False
is ww1 cj? 4 False
is ww1 cj? 5 False
is ww1 cj? 6 False
is ww1 cj? 7 False
is ww1 cj? 8 False
is ww1 cj? 9 False
is ww1 cj? 10 False
is ww1 cj? 11 False
is ww1 cj? 12 False
is ww1 cj? 13 False
is ww1 cj? 14 False
is ww1 cj? 15 False
is ww1 cj? 16 False
https://mempool.space/tx/57dfe22896ee38e71f064ccbe8fca668f5d82aa1d437736b513e4f3f8e55d845
76782bb6741e6198ef6cb0ff99debf2216f4e919ba7ce37c12657d0af89e9ee6
116b1f585592080f78107a4b5493910cfda9a5bd215d15585dd8a5cc1a8f1700
d82f1bf8ceee3892b95f8be6b559ab1674e7a9373564ee5fe4814338fe267d5a
CoinJoinType.WW1
is ww1 cj? 0 False
is ww1 cj? 1 False
is ww1 cj? 2 False
is ww1 cj? 3 False
is ww1 cj? 4 False
is ww1 cj? 5 False
is ww1 cj? 6 True
is ww1 cj? 7 False
is ww1 cj? 8 False
is ww1 cj? 9 True
is ww1 cj? 10 False
https://mempool.space/tx/d82f1bf8ceee3892b95f8be6b559ab1674e7a9373564ee5fe4814

In [3]:
"""
Table structure
cj pool | operating period | total cjtxs | fresh inputs | remix rate | unmoved UTXOs | num of inputs (min, avg, max)
ww1
ww2 (zkSNACKs)
ww2 
wp all
wp 5m
wp 1m
wp 50m
wp 100k
"""
april_24_block_height = get_block_height_for_date('2024-04-24')
june_1_block_height = get_block_height_for_date('2024-06-01')
last_august_block_height = get_block_height_for_date('2024-08-31')

In [4]:
%time ww2_cjs = chain.filter_coinjoin_txes(0, june_1_block_height, "ww2")
%time ww2_cjs_hashes = set(str(tx.hash) for tx in ww2_cjs)

CPU times: user 1min 4s, sys: 43.7 s, total: 1min 48s
Wall time: 1.07 s
CPU times: user 58 ms, sys: 83.4 ms, total: 141 ms
Wall time: 142 ms


In [5]:
%time ww2_cjs_other = chain.filter_coinjoin_txes(june_1_block_height, last_august_block_height, "ww2")
%time ww2_cjs_other_hashes = set(str(tx.hash) for tx in ww2_cjs_other)

CPU times: user 4.05 s, sys: 2.67 s, total: 6.72 s
Wall time: 113 ms
CPU times: user 1.93 ms, sys: 13 ms, total: 15 ms
Wall time: 14.5 ms


In [6]:
%time wp_cjs = chain.filter_coinjoin_txes(0, april_24_block_height, "wp")
%time wp_cjs_hashes = set(str(tx.hash) for tx in wp_cjs)

CPU times: user 1min 33s, sys: 248 ms, total: 1min 34s
Wall time: 771 ms
CPU times: user 663 ms, sys: 387 ms, total: 1.05 s
Wall time: 1.05 s


In [7]:
%time ww1_cjs = chain.filter_coinjoin_txes(0, june_1_block_height, "ww1")
%time ww1_cjs_hashes = set(str(tx.hash) for tx in ww1_cjs) - ww1_false

CPU times: user 16min 42s, sys: 5.88 s, total: 16min 48s
Wall time: 27.3 s
CPU times: user 38.3 ms, sys: 38.8 ms, total: 77.1 ms
Wall time: 76.8 ms


In [9]:
god_set_of_all_coinjoins = ww1_cjs_hashes | wp_cjs_hashes | ww2_cjs_hashes | ww2_cjs_other_hashes
len(ww1_cjs)

26781

In [10]:
import copy
import json

wp_stats = {
    "all_txes": 0,
    "all_inputs": 0,
    "all_inputs_value": 0,
    "fresh_inputs": 0,
    "fresh_inputs_value": 0,
    "all_outputs": 0,
    "remixed_outputs": 0,
    "unmoved_outputs": 0,
    "unmoved_outputs_value": 0,
    "traverse_inputs": 0,
    "traverse_inputs_value": 0
}

ww2_stats = copy.deepcopy(wp_stats)
ww2_other_stats = copy.deepcopy(wp_stats)
ww1_stats = copy.deepcopy(wp_stats)

whirlpool_stats = {
    "all": copy.deepcopy(wp_stats),
    "5m": copy.deepcopy(wp_stats),
    "1m": copy.deepcopy(wp_stats),
    "50m": copy.deepcopy(wp_stats),
    "100k": copy.deepcopy(wp_stats),
}

def get_pool(ws, tx):
    max_pool_value = max(x.value for x in tx.outputs)
    if max_pool_value == 5000000:
        return ws["5m"]
    if max_pool_value == 1000000:
        return ws["1m"]
    if max_pool_value == 50000000:
        return ws["50m"]
    if max_pool_value == 100000:
        return ws["100k"]

    raise ValueError(f"wtf {max_pool_value}")

def check_fdnp(tx: blocksci.Tx) -> int:
    s = 0
    
    for i in tx.inputs:
        if str(i.spent_tx.hash) in ww2_cjs_hashes or str(i.spent_tx.hash) in ww2_cjs_other_hashes:
            s += i.value

    return s


def should_ignore_input(tx):
    s = 0
    rv = False
    for i in tx.inputs:
        sp_tx = i.spent_tx
        if str(sp_tx.hash) in ww1_cjs_hashes:
            s += i.value
            rv = True
        elif str(i.spent_tx.hash) in ww2_cjs_hashes or str(i.spent_tx.hash) in ww2_cjs_other_hashes:
            s += i.value
            rv = True
    return s, rv

from collections import defaultdict
from typing import Dict, Set, List


def get_stats_for_cj(
    the_stats: Dict[str, int], 
    tx_objects: List[blocksci.Tx], 
    given_cjs: Set[str], 
    is_whirlpool: bool = False, 
    is_ww2: bool = False, 
    is_ww1: bool = False
):
    tx: blocksci.Tx
    tx_input: blocksci.Input
    tx_output: blocksci.Output
    dd_int = defaultdict(int)

    for tx in tx_objects:
        # filter out false positives
        if str(tx.hash) not in given_cjs:
            continue

        if is_whirlpool:
            tx_pool = get_pool(the_stats, tx)
            current_stats = the_stats["all"]
        else:
            tx_pool = dd_int
            current_stats = the_stats

        current_stats["all_txes"] += 1
        tx_pool["all_txes"] += 1
        
        current_stats["all_inputs"] += tx.input_count
        for tx_input in tx.inputs:
            current_stats["all_inputs_value"] += tx_input.value
            
            if str(tx_input.spent_tx.hash) not in given_cjs:
                how_much = 0
                ignore = False
                if is_ww1 or is_ww2:
                    how_much, ignore = should_ignore_input(tx_input.spent_tx)
  
                current_stats["fresh_inputs"] += (0 if ignore else 1) 
                tx_pool["fresh_inputs"] += 1 
                current_stats["fresh_inputs_value"] += (tx_input.value if not ignore else 0)
                tx_pool["fresh_inputs_value"] += tx_input.value

            if str(tx_input.spent_tx.hash) not in given_cjs and str(tx_input.spent_tx.hash) in god_set_of_all_coinjoins:
                current_stats["traverse_inputs"] += 1
                tx_pool["traverse_inputs"] += 1
                current_stats["traverse_inputs_value"] += tx_input.value
                tx_pool["traverse_inputs_value"] += tx_input.value

            if str(tx_input.spent_tx.hash) in given_cjs:
                current_stats["remixed_outputs"] += 1  
                tx_pool["remixed_outputs"] += 1  

        current_stats["all_outputs"] += tx.output_count
        tx_pool["all_outputs"] += tx.output_count
        for tx_output in tx.outputs:
            if not tx_output.is_spent:
                current_stats["unmoved_outputs"] += 1
                tx_pool["unmoved_outputs"] += 1
                current_stats["unmoved_outputs_value"] += tx_output.value
                tx_pool["unmoved_outputs_value"] += tx_output.value
                continue
                


In [11]:
%time get_stats_for_cj(whirlpool_stats, wp_cjs, wp_cjs_hashes, is_whirlpool=True)
%time get_stats_for_cj(ww2_stats, ww2_cjs, ww2_cjs_hashes, is_ww2=True)
%time get_stats_for_cj(ww2_other_stats, ww2_cjs_other, ww2_cjs_other_hashes, is_ww2=True)
%time get_stats_for_cj(ww1_stats, ww1_cjs, ww1_cjs_hashes, is_ww1=True)

CPU times: user 1min 48s, sys: 961 ms, total: 1min 49s
Wall time: 1min 49s
CPU times: user 2min 7s, sys: 971 ms, total: 2min 8s
Wall time: 2min 9s
CPU times: user 17.1 s, sys: 133 ms, total: 17.3 s
Wall time: 17.4 s
CPU times: user 45.5 s, sys: 435 ms, total: 46 s
Wall time: 46.3 s


In [12]:
from tabulate import tabulate

In [13]:
print(tabulate(
    [
        [
            "Wasabi 1.0 (zkSNACKs)", 
            "2018-07-19 – 2024-06-01", 
            len(ww1_cjs_hashes), 
            f"{ww1_stats['fresh_inputs'] // 1000}k/{round(ww1_stats['fresh_inputs_value'] / 100000000, 1)} BTC", 
            f"{round((ww1_stats['remixed_outputs'] / ww1_stats['all_outputs']) * 100, 1)}%", 
            f"{round((ww1_stats['unmoved_outputs'] / (ww1_stats['all_outputs'] - ww1_stats['remixed_outputs'])) * 100, 1)}%, {round(ww1_stats['unmoved_outputs_value'] / 100000000, 0)}"
        ],
        [
            "Wasabi 2.0 (zkSNACKs)", 
            "2022-06-18 – 2024-06-01", 
            len(ww2_cjs_hashes), 
            f"{ww2_stats['fresh_inputs'] // 1000}k/{round(ww2_stats['fresh_inputs_value'] / 100000000, 0)} BTC", 
            f"{round((ww2_stats['remixed_outputs'] / ww2_stats['all_outputs']) * 100, 1)}%", 
            f"{round((ww2_stats['unmoved_outputs'] / (ww2_stats['all_outputs'] - ww2_stats['remixed_outputs'])) * 100, 1)}%, {round(ww2_stats['unmoved_outputs_value'] / 100000000, 0)}"        
        ],
        [
            "Wasabi 2.0 (other)", 
            "2024-06-01 – current", 
            len(ww2_cjs_other_hashes), 
            f"{ww2_other_stats['fresh_inputs'] // 1000}k/{round(ww2_other_stats['fresh_inputs_value'] / 100000000, 0)} BTC", 
            f"{round((ww2_other_stats['remixed_outputs'] / ww2_other_stats['all_outputs']) * 100, 1)}%", 
            f"{round((ww2_other_stats['unmoved_outputs'] / (ww2_other_stats['all_outputs'] - ww2_other_stats['remixed_outputs'])) * 100, 1)}%, {round(ww2_other_stats['unmoved_outputs_value'] / 100000000, 0)}"        
        ],
        *(
            [
                f"Whirlpool {key}", 
                "2019-04-17 – 2024-04-24", 
                whirlpool_stats[key]["all_txes"], 
                f"{whirlpool_stats[key]['fresh_inputs'] // 1000}k/{round(whirlpool_stats[key]['fresh_inputs_value'] / 100000000, 1)} BTC", 
                f"{round((whirlpool_stats[key]['remixed_outputs'] / max(whirlpool_stats[key]['all_outputs'], 1)) * 100, 1)}%", 
                f"{round(whirlpool_stats[key]['unmoved_outputs'] / (max((whirlpool_stats[key]['all_outputs'] - whirlpool_stats[key]['remixed_outputs']), 1)) * 100, 1)}%, {round(whirlpool_stats[key]['unmoved_outputs_value'] / 100000000, 0)}"        
            ] for key in whirlpool_stats
        )
        
    ],
    headers=['Pool', 'Operating period', "Total cjtxs", "Fresh inputs", "Remix rate", "Unmoved UTXOs"], 
    tablefmt='orgtbl'
    )
)


| Pool                  | Operating period        |   Total cjtxs | Fresh inputs      | Remix rate   | Unmoved UTXOs   |
|-----------------------+-------------------------+---------------+-------------------+--------------+-----------------|
| Wasabi 1.0 (zkSNACKs) | 2018-07-19 – 2024-06-01 |         26600 | 282k/177865.0 BTC | 53.7%        | 3.2%, 3241.0    |
| Wasabi 2.0 (zkSNACKs) | 2022-06-18 – 2024-06-01 |         34742 | 272k/71546.0 BTC  | 86.0%        | 5.9%, 6314.0    |
| Wasabi 2.0 (other)    | 2024-06-01 – current    |          2539 | 17k/3232.0 BTC    | 74.0%        | 20.4%, 1367.0   |
| Whirlpool all         | 2019-04-17 – 2024-04-24 |        541011 | 1134k/47291.4 BTC | 61.0%        | 10.6%, 8411.0   |
| Whirlpool 5m          | 2019-04-17 – 2024-04-24 |        112100 | 233k/11702.6 BTC  | 61.4%        | 10.9%, 1271.0   |
| Whirlpool 1m          | 2019-04-17 – 2024-04-24 |        205626 | 434k/4359.3 BTC   | 60.1%        | 8.8%, 384.0     |
| Whirlpool 50m         | 2019-0

In [58]:
from typing import Dict
data: Dict[str, blocksci.address_type] = {}



df = pd.DataFrame(data)
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Group by timeframe and script type
grouped = df.groupby([df['timestamp'], 'script_type']).size().reset_index(name='count')

# Pivot to get counts of each script type per timestamp
pivot_df = grouped.pivot(index='timestamp', columns='script_type', values='count').fillna(0)

# Calculate total transactions per timestamp and percentages
pivot_df['total'] = pivot_df.sum(axis=1)
percentages = pivot_df.div(pivot_df['total'], axis=0) * 100
percentages = percentages.drop(columns=['total'])

# Plotting
plt.figure(figsize=(12, 6))
percentages.plot(kind='line', marker='o', ax=plt.gca())
plt.title('Bitcoin Script Type Usage Over Time')
plt.xlabel('Date')
plt.ylabel('Percentage')
plt.legend(title='Script Type')
plt.grid()
plt.show()



0 0
